In [1]:
import pickle
import h5py
import os
from sklearn import svm, tree
from statistics import mean
from random import shuffle
import numpy as np
from sklearn.utils import shuffle as shf
from sklearn.neural_network import MLPClassifier
import graphviz

In [2]:
data2 = {}
print('loading data')
for key in os.listdir('E:/Time Series/hdf_files2'):
	filename = 'E:/Time Series/hdf_files2/'+ key
	f = h5py.File(filename, 'r')
	data2[key] = list(list(f.values())[0])
	f.close()

loading data


In [3]:
data = data2

In [4]:
for key, value in data2.items():
    print(key, '---', len(value))
    shuffle(data2[key])

ALFALFA --- 20000
CORN --- 20000
DEVELOPED --- 20000
SOYABEENS --- 20000
WATER_BODY --- 20000


In [5]:
data = {'DEVELOPED':data2['DEVELOPED'][:19500], 'ALFALFA':data2['ALFALFA'][:2000], 'CORN': data2['CORN'][:5000], 'SOYABEENS':data2['SOYABEENS'][:500], 'WATERBODY':data2['WATER_BODY'][:3000]}

In [6]:
data_cv = {'DEVELOPED':data2['DEVELOPED'][19500:], 'ALFALFA':data2['ALFALFA'][19500:], 'CORN': data2['CORN'][19500:], 'SOYABEENS':data2['SOYABEENS'][19900:], 'WATERBODY':data2['WATER_BODY'][19500:]}

In [5]:
data = {key:value[:19000] for key, value in data2.items() if len(value) >= 20000}

In [7]:
mins = {}; maxs = {}; means = {}
for i in range(45):
    lin = [l[i] for value in data.values() for l in value]
    mins[i] = min(lin)
    maxs[i] = max(lin)
    means[i] = mean(lin)

In [8]:
data = {key: [[np.float64((example[i]-means[i])/(maxs[i] - mins[i])) for i in range(45)] for example in value] for key, value in data.items()}

C:\Users\prabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in ushort_scalars
  """Entry point for launching an IPython kernel.


In [9]:
for key in data.keys():
    shuffle(data[key])

In [10]:
training_examples = []
training_labels = []
i = 0;
for key, value in data.items():
    training_examples = training_examples + value
    training_labels = training_labels + [i for v in range(len(value))]
    print(i,':', key)
    i+=1

0 : DEVELOPED
1 : ALFALFA
2 : CORN
3 : SOYABEENS
4 : WATERBODY


In [11]:
training_examples, training_labels  = shf(training_examples, training_labels)

In [23]:
data_cv = {key:value[19000:] for key, value in data2.items() if len(value) >= 20000}

In [12]:
data_cv = {key: [[np.float64((example[i]-means[i])/(maxs[i] - mins[i])) for i in range(45)] for example in value] for key, value in data_cv.items()}

C:\Users\prabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in ushort_scalars
  """Entry point for launching an IPython kernel.


In [13]:
cv_examples = []
cv_labels = []
i = 0;
for key, value in data_cv.items():
    cv_examples = cv_examples + value
    cv_labels = cv_labels + [i for v in range(len(value))]
    print(i,':', key)
    i+=1

0 : DEVELOPED
1 : ALFALFA
2 : CORN
3 : SOYABEENS
4 : WATERBODY


In [14]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
be = SVC(C=1.0, 
         cache_size=200, 
         class_weight=None, 
         coef0=0.0,
         decision_function_shape='ovr', 
         degree=3, 
         gamma='auto',
         kernel='rbf',
         max_iter=-1, 
         probability=False, 
         random_state=None, 
         shrinking=True, 
         tol=0.001, 
         verbose=False)
crf = BaggingClassifier(base_estimator=be,
                        n_estimators=10,
                        max_samples=0.,
                        max_features = 0.5,
                        bootstrap=True,
                        bootstrap_features=False,
                        oob_score=True,
                        warm_start=False,
                        n_jobs=-1,
                        random_state=None,
                        verbose=0)
crf = crf.fit(training_examples, training_labels)
print(sum(crf.predict(training_examples)==training_labels)/len(training_labels))
i = 0
cv_predictions = []
for key, value in data_cv.items():
    t = crf.predict(value)
    print(sum(t==[i for t in value])/len(value), key)
    cv_predictions = cv_predictions + list(t)
    i = i+1
print(sum(np.array(cv_predictions) == cv_labels)/len(cv_labels))
crf.base_estimator_

C:\Users\prabh\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:605: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\prabh\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:610: RuntimeWarning: invalid value encountered in true_divide
  predictions.sum(axis=1)[:, np.newaxis])


0.7485333333333334
0.974 DEVELOPED
0.06 ALFALFA
0.304 CORN
0.0 SOYABEENS
0.562 WATERBODY
0.4523809523809524


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [17]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
be = SVC(C=1.0, 
         cache_size=200, 
         class_weight=None, 
         coef0=0.0,
         decision_function_shape='ovr', 
         degree=3, 
         gamma='auto',
         kernel='rbf',
         max_iter=-1, 
         probability=True, 
         random_state=None, 
         shrinking=True, 
         tol=0.001, 
         verbose=False)
crf = BaggingClassifier(base_estimator=be,
                        n_estimators=10,
                        max_samples=0.3,
                        max_features = 0.5,
                        bootstrap=True,
                        bootstrap_features=False,
                        oob_score=True,
                        warm_start=False,
                        n_jobs=-1,
                        random_state=None,
                        verbose=0)
crf = crf.fit(training_examples, training_labels)
print(sum(crf.predict(training_examples)==training_labels)/len(training_labels))
i = 0
cv_predictions = []
for key, value in data_cv.items():
    t = crf.predict(value)
    print(sum(t==[i for t in value])/len(value), key)
    cv_predictions = cv_predictions + list(t)
    i = i+1
print(sum(np.array(cv_predictions) == cv_labels)/len(cv_labels))
crf.base_estimator_

0.7636
0.956 DEVELOPED
0.228 ALFALFA
0.328 CORN
0.0 SOYABEENS
0.596 WATERBODY
0.5019047619047619


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [11]:
clf = svm.SVC(C=100000,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.0000001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
sum(clf.predict(training_examples) == training_labels)/len(training_labels)

[LibSVM]

0.973825

In [12]:
import pickle
file_object = open('9_pixels_learning_2.pkl', 'wb')
pickle.dump([mins, maxs, means, clf], file_object)
file_object.close()

In [15]:
from sklearn.ensemble import RandomForestClassifier
crf = RandomForestClassifier(n_estimators=10, 
                             criterion='gini', 
                             max_depth=None, 
                             min_samples_split=2, 
                             min_samples_leaf=1, 
                             min_weight_fraction_leaf=0.0, 
                             max_features='auto', 
                             max_leaf_nodes=None, 
                             min_impurity_decrease=0.0, 
                             min_impurity_split=None, 
                             bootstrap=True, 
                             oob_score=False, 
                             n_jobs=1, 
                             random_state=None, 
                             verbose=0, 
                             warm_start=False, 
                             class_weight=None)
crf = crf.fit(training_examples, training_labels)
print(sum(crf.predict(training_examples)==training_labels)/len(training_labels))
i = 0
for key, value in data_cv.items():
    print(sum(crf.predict(value)==[i for t in value])/len(value), key)
    i = i+1

0.9919666666666667
0.964 DEVELOPED
0.454 ALFALFA
0.742 CORN
0.24 SOYABEENS
0.76 WATERBODY


In [35]:
from sklearn.ensemble import RandomForestClassifier
crf = RandomForestClassifier(n_estimators=10, 
                             criterion='gini', 
                             max_depth=30, 
                             min_samples_split=2, 
                             min_samples_leaf=1, 
                             min_weight_fraction_leaf=0.0, 
                             max_features='auto', 
                             max_leaf_nodes=None, 
                             min_impurity_decrease=0.0, 
                             min_impurity_split=None, 
                             bootstrap=True, 
                             oob_score=False, 
                             n_jobs=1, 
                             random_state=None, 
                             verbose=0, 
                             warm_start=False, 
                             class_weight={0:1/19500, 1: 1/2000, 2:1/5000 ,3:1/500 ,4:1/3000})
crf = crf.fit(training_examples, training_labels)
print(sum(crf.predict(training_examples)==training_labels)/len(training_labels))
i = 0
cv_predictions = []
for key, value in data_cv.items():
    t = crf.predict(value)
    print(sum(t==[i for t in value])/len(value), key)
    cv_predictions = cv_predictions + list(t)
    i = i+1
print(sum(np.array(cv_predictions) == cv_labels)/len(cv_labels))

0.9902
0.958 DEVELOPED
0.426 ALFALFA
0.74 CORN
0.21 SOYABEENS
0.764 WATERBODY
0.6976190476190476


In [37]:
from sklearn.tree import ExtraTreeClassifier
crf = ExtraTreeClassifier(criterion='gini', splitter='random', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None)
crf = crf.fit(training_examples, training_labels)
print(sum(crf.predict(training_examples)==training_labels)/len(training_labels))
i = 0
cv_predictions = []
for key, value in data_cv.items():
    t = crf.predict(value)
    print(sum(t==[i for t in value])/len(value), key)
    cv_predictions = cv_predictions + list(t)
    i = i+1
print(sum(np.array(cv_predictions) == cv_labels)/len(cv_labels))

SyntaxError: invalid character in identifier (<ipython-input-37-4ac2fa5a07c2>, line 2)

In [33]:
sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels)

0.8318

In [14]:
ctf = tree.DecisionTreeClassifier()
ctf = ctf.fit(training_examples, training_labels)
print(sum(ctf.predict(cv_examples)==cv_labels)/len(cv_labels))
print(sum(ctf.predict(training_examples)==training_labels)/len(training_labels))
import graphviz 
dot_data = tree.export_graphviz(ctf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("decisiontree2") 

0.7676
1.0


'decisiontree2.pdf'

In [28]:
ctf = tree.DecisionTreeClassifier(max_depth = 3)
ctf = ctf.fit(training_examples, training_labels)
print(sum(ctf.predict(cv_examples)==cv_labels)/len(cv_labels))
print(sum(ctf.predict(training_examples)==training_labels)/len(training_labels))
'''import graphviz 
dot_data = tree.export_graphviz(ctf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("decisiontree2") '''

0.598
0.6069157894736842


'import graphviz \ndot_data = tree.export_graphviz(ctf, out_file=None) \ngraph = graphviz.Source(dot_data) \ngraph.render("decisiontree2") '

In [29]:
ctf = tree.DecisionTreeClassifier(max_depth = 4)
ctf = ctf.fit(training_examples, training_labels)
print(sum(ctf.predict(cv_examples)==cv_labels)/len(cv_labels))
print(sum(ctf.predict(training_examples)==training_labels)/len(training_labels))
import graphviz 
dot_data = tree.export_graphviz(ctf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("decisiontree4")

0.6362
0.6449263157894737


'decisiontree4.pdf'

In [31]:
ctf = tree.DecisionTreeClassifier(max_depth = 5)
ctf = ctf.fit(training_examples, training_labels)
print(sum(ctf.predict(cv_examples)==cv_labels)/len(cv_labels))
print(sum(ctf.predict(training_examples)==training_labels)/len(training_labels))
import graphviz 
dot_data = tree.export_graphviz(ctf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("decisiontree5")

0.695
0.695178947368421


'decisiontree5.pdf'

In [32]:
ctf = tree.DecisionTreeClassifier(max_depth = 6)
ctf = ctf.fit(training_examples, training_labels)
print(sum(ctf.predict(cv_examples)==cv_labels)/len(cv_labels))
print(sum(ctf.predict(training_examples)==training_labels)/len(training_labels))
import graphviz 
dot_data = tree.export_graphviz(ctf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("decisiontree6")

0.7138
0.7228105263157895


'decisiontree6.pdf'

In [33]:
ctf = tree.DecisionTreeClassifier(max_depth = 7)
ctf = ctf.fit(training_examples, training_labels)
print(sum(ctf.predict(cv_examples)==cv_labels)/len(cv_labels))
print(sum(ctf.predict(training_examples)==training_labels)/len(training_labels))
import graphviz 
dot_data = tree.export_graphviz(ctf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("decisiontree7")

0.7398
0.7406947368421053


'decisiontree7.pdf'

In [34]:
ctf = tree.DecisionTreeClassifier(max_depth = 8)
ctf = ctf.fit(training_examples, training_labels)
print(sum(ctf.predict(cv_examples)==cv_labels)/len(cv_labels))
print(sum(ctf.predict(training_examples)==training_labels)/len(training_labels))
import graphviz 
dot_data = tree.export_graphviz(ctf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("decisiontree8")

0.7474
0.7565052631578948


'decisiontree8.pdf'

In [37]:
ctf = tree.DecisionTreeClassifier(max_depth = 11)
ctf = ctf.fit(training_examples, training_labels)
print(sum(ctf.predict(cv_examples)==cv_labels)/len(cv_labels))
print(sum(ctf.predict(training_examples)==training_labels)/len(training_labels))
import graphviz 
dot_data = tree.export_graphviz(ctf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("decisiontree11")

0.7772
0.814


'decisiontree11.pdf'

In [17]:
clf = svm.SVC(C= 30000,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.0000001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.969625
0.77355


In [18]:
clf = svm.SVC(C= 10000,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.000001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.964525
0.7797


In [18]:
cv_examples = []
cv_labels = []
i = 0;
for key, value in data_cv.items():
    training_examples = training_examples + value[:4000]
    training_labels = training_labels + [i for v in range(4000)]
    cv_examples = cv_examples + value[4000:]
    cv_labels = cv_labels + [i for v in range(1000)]
    print(i,':', key)
    i+=1
training_examples, training_labels = shf(training_examples, training_labels)

0 : CORN
1 : DEVELOPED
2 : SOYABEENS
3 : WATER_BODY


In [21]:
clf = svm.SVC(C= 3000,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.00001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.9539821428571429
0.79575


In [21]:
clf = svm.SVC(C= 3000,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.00001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.9539821428571429
0.79575


In [22]:
clf = svm.SVC(C= 1000,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.9438392857142858
0.80225


In [20]:
clf = svm.SVC(C= 1,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.8666785714285714
0.7925


In [21]:
clf = svm.SVC(C= 3,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.8903571428571428
0.796


In [22]:
clf = svm.SVC(C= 10,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.9015357142857143
0.79925


In [23]:
clf = svm.SVC(C= 30,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.0000001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.9110357142857143
0.80475


In [24]:
clf = svm.SVC(C= 100,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.0000001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.9215
0.8085


In [29]:
clf = svm.SVC(C= 170,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.01, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.9262321428571428
0.80975


In [31]:
clf = svm.SVC(C= 1,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.01, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.8666607142857143
0.79275


In [42]:
clf = svm.SVC(C= 170,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.01, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.9264210526315789
0.80775


In [46]:
clf = svm.SVC(C= 2,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.000001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.8846842105263157
0.793


In [47]:
clf = svm.SVC(C= 3,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.000001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.8905921052631579
0.79175


In [25]:
clf = svm.SVC(C= 3,
              kernel='sigmoid', 
              degree= 2,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.000001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
print (sum(clf.predict(training_examples) == training_labels)/len(training_labels))
print(sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels))

[LibSVM]0.22557894736842105
0.22425


In [36]:
clf = svm.SVC(C= 3,
              kernel='rbf', 
              degree= 2,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.000001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)
clf.fit(training_examples, training_labels)
sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels)
sum(clf.predict(training_examples) == training_labels)/len(training_labels)

[LibSVM]

0.8432315789473684

In [15]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(training_examples, training_labels)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [17]:
s = model.feature_importances_
L = np.array(s)
t = sorted(range(len(L)), key=lambda i:L[i])
z = []; r = 0
for j in t[::-1]:
    print( j, '-----', s[j])
    if s[j]>0.02:
        z.append(j)
print(z)

44 ----- 0.0629666574050501
29 ----- 0.042415007033263896
9 ----- 0.04175524834329842
38 ----- 0.04035329115930774
13 ----- 0.030888461304013903
19 ----- 0.030191824894230872
14 ----- 0.029834342793438363
10 ----- 0.0288195506991518
0 ----- 0.02864051036691719
34 ----- 0.02692124937730108
18 ----- 0.02646967599997988
1 ----- 0.025950561874806842
8 ----- 0.024935955823798338
4 ----- 0.023591169984353032
43 ----- 0.022740660178468697
11 ----- 0.02206169851263916
23 ----- 0.021553974804947487
41 ----- 0.02113211872347874
26 ----- 0.02079389961651347
33 ----- 0.019670888829301285
37 ----- 0.019628781424271935
39 ----- 0.019118838662844466
28 ----- 0.01879564109780771
15 ----- 0.018755689470142032
7 ----- 0.018694204772217074
36 ----- 0.018208041329346347
32 ----- 0.01788341135447162
5 ----- 0.017631936986604858
30 ----- 0.017291377692063147
25 ----- 0.016395993392518276
24 ----- 0.01604944109013577
2 ----- 0.01602912643977806
20 ----- 0.01599004957051735
42 ----- 0.015989098124278456
16 --

In [26]:
training_examples = [[example[t] for t in z] for example in training_examples ]
cv_examples = [[example[t] for t in z]for example in cv_examples]

In [ ]:
cnf = MLPClassifier(hidden_layer_sizes=(1000,1000,1000), activation='tanh', solver='sgd',
                    alpha=0.00000001,   batch_size= 10000,    learning_rate='adaptive',
                    learning_rate_init=0.0001,    power_t=0.5,    max_iter=10000, 
                    shuffle=True,     random_state=None,  tol=0.00000001, 
                    verbose=True,     warm_start= True,   momentum=0.9, 
                    nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, 
                    beta_1=0.5, 
                    beta_2=0.5, 
                    epsilon=1e-10
                   )
cnf.fit(training_examples, training_labels)
sum(cnf.predict(training_examples) == training_labels)/len(training_labels)